In [ ]:
# Dimensiunea mediului
width = 2
height = 1

# Inițial, întreg spațiul este murdar.
env = [[1 for x in range(width)] for y in range(height)]

start = (0, 0)
# env[start[1]][start[0]] = 0

In [ ]:
# Am pus 'Clean' primul pentru ca eu parcurg `moves` in ordine si as fi mers pe `Right` inaite de `Clean`,
# deci `Clean` n-ar mai fi fost nedeterminist pentru ca pozitia ar fi fost la capatul din dreapta, iar secventa de
moves = ['Clean', 'Left', 'Right']

# efect is a tuple of:
#  delta-x
#  delta-y
#  cleanness of current cell if current cell was clean
#  cleanness of cell to the right if current cell was clean
#  cleanness of current cell if current cell was dirty
#  cleanness of cell to the right if current cell was dirty

# deterministic effects:
effectD = {}
effectD['Left'] = [(-1, 0, -1, -1, -1, -1)]
effectD['Right'] = [(+1, 0, -1, -1, -1, -1)]
effectD['Clean'] = [(0, 0, 0, -1, 0, -1)]

# non-deterministic effects:
effectN = {}
effectN['Left'] = effectD['Left']
effectN['Right'] = effectD['Right']
effectN['Clean'] = [(0, 0, 0, -1, 0, -1), (0, 0, 1, -1, 0, 0)]

In [ ]:
import operator
from functools import reduce

# Întoarce adevărat dacă celula este o celula în interiorul spațiului.
def is_good(state):
    return state[0] >= 0 and state[0] < width and state[1] >= 0 and state[1] < height

# Întoarce adevărat dacă toate celulele din mediu sunt curate.
def env_clean(env):    return 0 == len(list(filter(lambda x: x > 0, reduce(operator.add, env, []))))

# Întoarce o listă de tupluri (stare-nouă, mediu-nou), conținând ca singur element efectul
#    realizării mutării deterministe specificate. Dacă mutarea nu poate fi realizată, lista este nulă.
# Mediul întors este o copie (instanță nouă) a parametrului dat.
def compute_effectD(state, env, move):
    return compute_effects(state, env, move, effectD)

# Întoarce o listă de tupluri (stare-nouă, mediu-nou), conținând efectele realizării mutării nedeterministe specificate.
# Lista poate conține zero (dacă mutarea nu este posibilă), unul sau mai multe elemente distincte.
# Mediul întors este o copie (instanță nouă) a parametrului dat.
def compute_effectN(state, env, move):
    return compute_effects(state, env, move, effectN)

def compute_effects(state, env, move, effects):
    effects = [compute_effect(state, env, effect) for effect in effects[move]]
    effects = list(filter(lambda e: e is not None, effects))
    if len(effects) == 2 and effects[0] == effects[1]:
        return effects[:1]
    return effects

def compute_effect(state, env, effect):
    new_env = [line[:] for line in env]
    (x, y) = state
    new_state = tuple([state[idx] + effect[idx] for idx in range(2)])
    if not is_good(new_state):
        return None

    for d in range(2):
        clean_effect = effect[2 + d + env[y][x] * 2]
        if clean_effect >= 0 and is_good((x + d, y)):
            new_env[y][x + d] = clean_effect
    return (new_state, new_env)


printX = 1
print(env_clean(env))
print([compute_effectD((printX, 0), env, m) for m in  moves])
print(compute_effectD((printX, 0), env, 'Clean'))
print(compute_effectN((printX, 0), env, 'Clean'))

False
[[((1, 0), [[1, 0]])], [((0, 0), [[1, 1]])], []]
[((1, 0), [[1, 0]])]
[((1, 0), [[1, 0]])]


In [ ]:
TYPE = 0
STATE = 1
ENV = 2
CHILDREN = 3
TAG = 4
PATH = 5

%matplotlib inline
import matplotlib.pyplot as pyplot
import networkx as nx

counter = 0
labels = {}
nodes = []
edges = []


# reprezentăm un nod din arbore ca o listă
# [move, state, environment, children, tag(None/SUCCESS/LOOP), path]
# formată din mutarea realizată în nodul părinte, stare în urma mutării, starea mediului în urma mutării,
#   lista de copii ai nodului (tot noduri), etichetă, reprezentare a căii din rădăcină până în nod


# afișează un arbore format din noduri definite ca mai sus (se dă rădăcina arborelui, care conține și copiii, etc)
# parametrul onlyOR indică dacă arborele este format doar din noduri SAU (altfel, este interpretat ca arbore ȘI-SAU)
def printTree(root, onlyOR = True):
    # G = nx.Graph()

    printTreeEx(root, 0, onlyOR, None)

    # G.add_nodes_from(nodes)
    # G.add_edges_from(edges)
    # nx.draw(G)
    # pyplot.show() # display

def printTreeEx(node, indent, onlyOR, parent):
    global counter
    line = ""
    for i in range(indent):
        line += "   "
    if node[TYPE] == "OR":
        line += "|  "
        line += str(node[STATE]) + " : " + str(node[ENV])
    else:
        line += ". " + node[TYPE] + " -> "
        if onlyOR:
            line += str(node[STATE]) + " : " + str(node[ENV])
    if node[TAG] is not None:
        line += " " + node[TAG]
    print(line)
    counter += 1
    nodes.append(counter)
    if parent is not None:
        edges.append((parent, counter))
    labels[counter] = line
    for child in node[CHILDREN]:
        printTreeEx(child, indent + 1, onlyOR, node)

def printNode(node):
    tag = ""
    if node[TAG] is not None:
        tag = node[TAG]
    print(str(node[TYPE]) + " : " + str(node[STATE]) + " : " + str(node[ENV]) + " (" + str(len(node[CHILDREN])) + ") [" + tag + "]")

In [ ]:
def updateTags(node):
    children_tags = [updateTags(child_node) for child_node in node[CHILDREN]]

    if children_tags:
        if node[TYPE] == "OR":
            if "SOLVED" in children_tags:
                node[TAG] = "SOLVED"
            else:
                node[TAG] = "UNSOLVABLE"
        else:
            if "UNSOLVABLE" in children_tags:
                node[TAG] = "UNSOLVABLE"
            else:
                node[TAG] = "SOLVED"

    return node[TAG]

# Întoarce un arbore al căutării în spațiul env, pornind din starea start
def makeTree(start, env):
    root = ["OR", start, env, [], None, [(start, env)]]
    q = [root]

    while q:
        crt_node = q.pop(0)

        if crt_node[TYPE] == "OR":
            for move in moves:
                child_node = [move, crt_node[STATE], crt_node[ENV], [], None, crt_node[PATH]]
                child_effect = compute_effectN(child_node[STATE], child_node[ENV], child_node[TYPE])

                if child_effect:
                    q.append(child_node)
                    crt_node[CHILDREN].append(child_node)
        else:
            # pt ca efectul e nedeterminist trebuie ca TOATE efectele unui nod sa fie SUCCESS ca nodul sa fie SUCCESS
            for node_effect in compute_effectN(crt_node[STATE], crt_node[ENV], crt_node[TYPE]):
                child_node = ["OR", node_effect[0], node_effect[1], [], None, crt_node[PATH] + [node_effect]]
                crt_node[CHILDREN].append(child_node)

                if env_clean(child_node[ENV]):
                    child_node[TAG] = "SOLVED"
                elif node_effect in crt_node[PATH]:
                    child_node[TAG] = "UNSOLVABLE"
                else:
                    q.append(child_node)

    updateTags(root)
    return root

tree = makeTree(start, env)
#print(tree)
printTree(tree, False)

|  (0, 0) : [[1, 1]] SOLVED
   . Clean ->  SOLVED
      |  (0, 0) : [[0, 1]] SOLVED
         . Clean ->  UNSOLVABLE
            |  (0, 0) : [[0, 1]] UNSOLVABLE
            |  (0, 0) : [[1, 1]] UNSOLVABLE
         . Right ->  SOLVED
            |  (1, 0) : [[0, 1]] SOLVED
               . Clean ->  SOLVED
                  |  (1, 0) : [[0, 0]] SOLVED
               . Left ->  UNSOLVABLE
                  |  (0, 0) : [[0, 1]] UNSOLVABLE
      |  (0, 0) : [[0, 0]] SOLVED
   . Right ->  SOLVED
      |  (1, 0) : [[1, 1]] SOLVED
         . Clean ->  SOLVED
            |  (1, 0) : [[1, 0]] SOLVED
               . Clean ->  UNSOLVABLE
                  |  (1, 0) : [[1, 0]] UNSOLVABLE
                  |  (1, 0) : [[1, 1]] UNSOLVABLE
               . Left ->  SOLVED
                  |  (0, 0) : [[1, 0]] SOLVED
                     . Clean ->  SOLVED
                        |  (0, 0) : [[0, 0]] SOLVED
                     . Right ->  UNSOLVABLE
                        |  (1, 0) : [[1, 0]] UNSOL

In [ ]:
# Întoarce un plan de acțiuni care, conform arborelui ȘI-SAU dat, duc la realizarea scopului. Planul este textual.
# Exemplu: "Clean; if env is [0, 0] then [DONE]; if env is [0, 1] then [Right; Clean]"
def makePlan(node):
    if node[TAG] == "UNSOLVABLE":
        return
    if not node[CHILDREN]:
        return node[TAG]

    if node[TYPE] == "OR":
        for child_node in node[CHILDREN]:
            child_plan = makePlan(child_node)

            if child_plan:
                return f"{child_node[TYPE]}; {child_plan}"
    else:
        # plan => environment; logic ar fi fost invers pentru ca mediul determina planul,
        # dar Python nu stie sa hashuiasca liste... :(
        decisions = {}

        for child_node in node[CHILDREN]:
            child_plan = makePlan(child_node)

            if child_plan:
                decisions[child_plan] = " ".join(map(str, child_node[ENV]))

        if len(decisions) == 1:
            return list(decisions.keys())[0]
        if not decisions or len(decisions) != len(node[CHILDREN]):
            return
        else:
            return "; ".join(f"if ENV == {env}: {{ [{plan}] }}" for plan, env in decisions.items())

    return

print(makePlan(tree))

Clean; if ENV == [0, 1]: { [Right; Clean; SOLVED] }; if ENV == [0, 0]: { [SOLVED] }
